## Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

### Inputs required

* customers_FR.csv - Customers data after Feature Reduction
* unknown_values.csv - Levels that should be coded as NULL, from Data Dictionary
* mailout_train_out.csv - A subset of the train data with only the columns that have been found after FR on Customers data. ~60 features.
* mailout_test.csv - A subset of the test data with only the columns that have been found after FR on Customers data. ~60 features.

### Steps

1. Read in the data and drop object columns. Since low volumes (7) - we can just exclude these rather than having to encode. Although, so nominal values may be coded with numeric values.
2. Replace missings all with -1
3. Run RandomizedSearchCV to get an idea of what the good params might be then run Gridsearch on a RandomForestClassifier.
4. Score-up test data and output this to a csv for kaggle

### Install packages

In [1]:
import pandas as pd
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

In [2]:
# print versions
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

print('pandas')
print(pd.__version__)
print('sklearn')
# print(sklearn.__version__)

Python version
3.8.12 | packaged by conda-forge | (default, Oct 12 2021, 21:59:51) 
[GCC 9.4.0]
Version info.
sys.version_info(major=3, minor=8, micro=12, releaselevel='final', serial=0)
pandas
1.4.2
sklearn
1.0.1


### Data and Data prep

In [3]:
# read the training dataset in 
df1 = pd.read_csv('train1.csv', low_memory=False)
df2 = pd.read_csv('train2.csv', low_memory=False)
df = pd.concat([df1, df2], axis=0)

In [5]:
df.RESPONSE.value_counts(dropna=False)

0    42430
1      532
Name: RESPONSE, dtype: int64

In [6]:
# get rid of object columns 
df = df.select_dtypes(exclude=['object'])

In [7]:
# confirm we have no columns as objects
print(sum(df.dtypes == object))

0


In [8]:
# since no columns have negative values I will try imputing all nan with -1. The data dictionary suggests most are coded as -1 anyway.
df = df.fillna(-1)

In [9]:
# split the dataframe into X and y in preparation of model training
y = df.RESPONSE
X = df.drop('RESPONSE', axis=1)

In [10]:
# run a quick randomised search in order to get a feel of what the proper full grid search parameter grid should be 

param_grid = {'max_depth': [None, 3, 5, 7, 9],
 'max_features': ['auto', 10, 15, 20 , 25 ,30 ,35, 40, 45, 50, 55, 60, 70, 80, 100, 120, 150, 170, 200, 250],
 'n_estimators': [50, 100, 250, 500, 1000, 2000],
 'min_samples_split': [100, 250, 500, 1000],
 'bootstrap': [True, False],
 'class_weight': ['balanced', None]
 }

base_estimator = RandomForestClassifier(random_state=2022, verbose=1, n_jobs=-1)

rs = RandomizedSearchCV(base_estimator, param_grid, cv=3, scoring='roc_auc', verbose=1, n_iter=50).fit(X, y)
rs.best_params_

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 704 tasks      | elapsed:  6.5min
[Parallel(n_jobs=-1)]: Done 1154 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 1704 tasks      | elapsed:  6.7min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:  6.8min finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:    0.2s
[Parallel(n_jobs=48)]: Done 704 tasks      | elapsed:    0.3s
[Parallel(n_jobs=48)]: Done 1154 tasks      | elapsed:    0.4s
[Parallel(n_jobs=48)]: Done 1704 tasks      | elapsed:    0.5s
[Parallel(n_jobs=48)]: Done 2000 out of 2000 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent wor

[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:    0.2s
[Parallel(n_jobs=48)]: Done 704 tasks      | elapsed:    0.3s
[Parallel(n_jobs=48)]: Done 1154 tasks      | elapsed:    0.4s
[Parallel(n_jobs=48)]: Done 1704 tasks      | elapsed:    0.5s
[Parallel(n_jobs=48)]: Done 2000 out of 2000 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 612 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    3.7s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 704 tasks      | elapsed:    0.2s
[Parallel(n_jobs=48)]: Done 1000 out of 1000 | elapsed:    0.3s finished


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  50 | elapsed:    0.4s remaining:    3.1s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.7s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done   6 out of  50 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=48)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  50 | elapsed:    0.4s remaining:    3.0s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.7s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done   6 out of  50 | elapsed:    0.0s remaining:    0.1s
[Parallel(n_jobs=48)]: Done  50 out of  50 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent worker

[Parallel(n_jobs=48)]: Done 704 tasks      | elapsed:    0.2s
[Parallel(n_jobs=48)]: Done 1000 out of 1000 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.1s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.1s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=48)]: D

...

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.8s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=48)]: Done 250 out of 250 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.8s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    0.

[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.7s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=48)]: Done 250 out of 250 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.7s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 250 out of 250 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed:    0.7s finished
[Parallel(n_jobs=

[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 500 out of 500 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.6s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.7s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 100 out

[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 704 tasks      | elapsed:    0.2s
[Parallel(n_jobs=48)]: Done 1000 out of 1000 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 612 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 1312 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:    5.6s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 704 tasks      | elapsed:    0.2s
[Parallel(n_jobs=48)]: Done 1154 tasks      | elapsed:    0.3s
[Parallel(n_jobs=48)]: Done 1704 tasks      | elapsed:    0.4s
[Parallel(n_jobs=48)]: Done 2000 out of 2000 | elapsed:    0.5s finished

{'n_estimators': 2000,
 'min_samples_split': 250,
 'max_features': 250,
 'max_depth': 3,
 'class_weight': 'balanced',
 'bootstrap': True}

In [12]:
# run the full grid search - I've changed the options here based on the above
new_param_grid = {'max_depth': [None, 3, 5, 9],
 'max_features': ['auto', 30, 60, 80, 100, 250],
 'n_estimators': [500, 1000, 1500, 2000],
 'min_samples_split': [200, 250, 500],
 'bootstrap': [True],
 'class_weight': ['balanced']
 }

gs = GridSearchCV(base_estimator, new_param_grid, cv=3, scoring='roc_auc', verbose=2).fit(X, y)
gs.best_params_

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.0s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 500 out of 500 | elapsed:    0.1s finished


[CV] END bootstrap=True, class_weight=balanced, max_depth=None, max_features=auto, min_samples_split=200, n_estimators=500; total time=   1.4s


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.0s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 500 out of 500 | elapsed:    0.1s finished


[CV] END bootstrap=True, class_weight=balanced, max_depth=None, max_features=auto, min_samples_split=200, n_estimators=500; total time=   1.5s


...

[CV] END bootstrap=True, class_weight=balanced, max_depth=9, max_features=250, min_samples_split=500, n_estimators=2000; total time=  29.3s


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 704 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 1154 tasks      | elapsed:   15.9s
[Parallel(n_jobs=-1)]: Done 1704 tasks      | elapsed:   23.3s
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:   27.1s finished
[Parallel(n_jobs=48)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=48)]: Done 104 tasks      | elapsed:    0.0s
[Parallel(n_jobs=48)]: Done 354 tasks      | elapsed:    0.1s
[Parallel(n_jobs=48)]: Done 704 tasks      | elapsed:    0.2s
[Parallel(n_jobs=48)]: Done 1154 tasks      | elapsed:    0.3s
[Parallel(n_jobs=48)]: Done 1704 tasks      | elapsed:    0.5s
[Parallel(n_jobs=48)]: Done 2000 out of 2000 | elapsed:    0.5s finished


[CV] END bootstrap=True, class_weight=balanced, max_depth=9, max_features=250, min_samples_split=500, n_estimators=2000; total time=  28.3s


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 48 concurrent workers.
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 354 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done 704 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   11.3s finished


{'bootstrap': True,
 'class_weight': 'balanced',
 'max_depth': 3,
 'max_features': 250,
 'min_samples_split': 200,
 'n_estimators': 1000}

In [13]:
# get the best parameters of the gridsearch and train our tuned model 
tuned_model = RandomForestClassifier(**gs.best_params_)
tuned_model.fit(X, y)

RandomForestClassifier(class_weight='balanced', max_depth=3, max_features=250,
                       min_samples_split=200, n_estimators=1000)

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter.

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [15]:
# read the training dataset in 
df1_test = pd.read_csv('test1.csv', low_memory=False)
df2_test  = pd.read_csv('test2.csv', low_memory=False)
df_test  = pd.concat([df1_test, df2_test], axis=0)
df_test.shape

(42833, 366)

In [16]:
# bring the test dataset in the exact same shape as the train one 
df_test = df_test.select_dtypes(exclude=['object'])
df_test = df_test.fillna(-1)

In [17]:
# understand which columns of the training dataframe are not in the test one 
print ('Columns in training but no in test')
for col in X.columns:
    if col not in df_test.columns:
        print(col)

print ('')
print ('Columns in test but no in training')
# and the vice versa
for colu in df_test.columns:
    if colu not in X.columns:
        print(colu)

Columns in training but no in test

Columns in test but no in training


In [18]:
# confirm that the column names and order match between the two dataframes
X.columns == df_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [19]:
# make sure that the order of the columns is the same
df_test = df_test.reindex(X.columns, axis=1)

In [20]:
# confirm that the column names and order match between the two dataframes
X.columns == df_test.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [21]:
# use the trained model to make predictions on the test dataframe
predictions = tuned_model.predict_proba(df_test)[:,1]

In [22]:
# isolate LNR
lnr = df_test['LNR'].copy()

# take predictions
results = pd.DataFrame({'LNR':lnr, 'RESPONSE':predictions})

In [23]:
# export results to csv
results.to_csv('KAGGLE_balanced_all2.csv', index=False)